# Proyecto Módulo 2: Implementación de un modelo de deep learning
##### David Armendáriz | A01570813

### Proposito:

El proposito de este proyecto es el de entrenar una red neuronal para identificar las placas de un carro. Aunque este proyecto se limita a detectar el objeto, la placa del carro, esto se puede llevar mas adelante para leer los contenidos de las placasa en si lo que se puede usar para diversas aplicaciones. Adicionalmente, previo a esto se podría implementar algun modelo que primero identifique un carro donde identificar una placa.

Para esto se busca emplear principalmente la técnica de transfer learning. Se implemento inicialmente un modelo de una simple red convolucional sequencial como un modelo de control para comparar los resultados del modelo pre-entrenado. Tras esto se re entrenó la red YOLOv11 sobre un dataset de placas de carros. Sobre el modelo YOLO generado, se varió el optimizador, las epocas y se implementó Data Augmentation. De todo esto se habla mas a detalle a continuación.

### Datos:
El dataset utilizado se obtuvo de kaggle. Este dataset contiene 433 imagenes en formato .png de carros con una o multiples placas visibles. Adicionalmente, se contienen archivos .xml para cada imagen con anotaciones correspondientes a las coordenadas de los vertices que deliminat las placas.

##### Preprocesamiento:
Todas las imagenes se escalaron a 224x224. Estas dimensiones se seleccionaron principalmente dados los estandares comunes del área de reconocimiento de imagenes, sin embargo, adicionalmete se buscaba un uso mas eficiente de recursos computacionales al reducir la dimendisón de las imagenes. Por ultimo, una razón importante del re escalado es el de mantener uniformidad a lo largo de los modelos.

Para la red neuronal de Control (Control_CNN.ipynb), el rescalado se realizo al momento de cargar las imagenes. Para las redes YOLO se usó la función format_data_yolo() en Image Handling.ipynb la cual modifico los archivos directamente y rescribio los archivos .xml con extensión .txt y formato YOLO (class x_center y_center box_width box_height).

Se reservó el 90% de las imagenes para entrenamiento, el 5% para validación y el 5% para prueba.

Por ultimo, se implementaron técnicas de data augmentation sobre 150 imagenes de entrenameinto. Se aplica aleatoriamente con una probablidad del 50% color jittering y/o se aplicó ruido sobre estas imagenes. Esto se hizo mediante la función augment_images() en Image Handling. De las 150 imagenes, para el set para la CNN de control se modifcaron 114 y del set para YOLO se modificaron 11. El subset de imagenes sujetas a la posiblidad de aumentarse se seleccionó de manera aleatoria mediante la función move_random_files() de Image Handling. Se seleccionaron solo tecnicas de data augmentation que no distorsionaran la forma de la placa debido a la limitante dada por las coordenadas predefeinidas en los archivos de los verticas de las placas los cuales serían dificiles de ajustar ante rotaciones, cortes, etc. debido a la naturaleza de los modelos de identificar objetos dentro de rectangulos sin rotación.


> Larxel. (2020). Car License Plate Detection. Kaggle. https://www.kaggle.com/datasets/andrewmvd/car-plate-detection/data


### Modelos:

##### Red CNN de Control (Control_CNN.ipynb):
Como ya se mención previamente el proposito principal de esta red es el de proporcionar resultados de control contra los cuales comparar el modelo YOLO. Dado esto, se generó una red de una arquitectura, parámetros y esquema de entrenamiento simple. A continuación se detalla la red.

| Layer Type          | Details                                     | Output Shape                |
|---------------------|---------------------------------------------|-----------------------------|
| Input Layer         | `input_shape = (height, width, channels)`  | `(height, width, channels)` |
| Convolutional Layer | `Conv2D(32, (3, 3), activation='relu')`     | `(height-2, width-2, 32)`   |
| Max Pooling Layer   | `MaxPooling2D((2, 2))`                     | `(height/2, width/2, 32)`   |
| Convolutional Layer | `Conv2D(64, (3, 3), activation='relu')`     | `(height/2-2, width/2-2, 64)`|
| Max Pooling Layer   | `MaxPooling2D((2, 2))`                     | `(height/4, width/4, 64)`   |
| Convolutional Layer | `Conv2D(128, (3, 3), activation='relu')`    | `(height/4-2, width/4-2, 128)`|
| Flattening Layer    | `Flatten()`                                | `(flattened_size)`          |
| Dense Layer         | `Dense(512, activation='relu')`            | `(512)`                     |
| Output Layer        | `Dense(4)`                                 | `(4)`                       |


Se implementaron 3 capas convolucionales, la intención de de la primera es extrar las caracteristicas más basicas de la imagen, idealmente los ejes de la placa o el vehiculo. Mientras que las otras dos extraen mas detalles. La primera capa convolucional es la de entrada que recibe las imagenes 224x224 rgb.

De manera similar, se implementaron 3 capas de Max Pooling para eficientizar los recursos computacionales empleados y reducir el posible overfit.

Se implementó una flattening layer para preparar los datos para la siguiente capa densa la cual se implementó con la intención de que aprendiera los patrones mas complejos.

La utlima capa densa de 4 outputs denota los 4 vertices de la bounding-boc de la placa.

Se diseño la red con simplicidad en mente. Esto fue la principal razón para la selección de la función de activación relu y el optimizador Adam ya que estos han probado ser efectivos en redes CNN y multiples otras tareas contando con un rapido y eficiente desempeño.

La función de loss, MSE se utilizó ya que es ideal para minimizar la distancia entre el valor predecido y el valor actual, lo que se ajusta a las necesidades de este proyecto.

##### Modelo You Only Look Once (YOLO.ipynb, YOLO_Tuned.ipynb):
Este es el modelo principal se re entrenó el modelo YOLO11 el cual es un modelo para la detección en tiempo real de objetos. La versión utilizada, 11, es la mas nuevo de Ultralytics y la de mejor desempeño. 

El modelo YOLO se seleccionó debido a su rápido funcionamiento con alto desempeño, en comparación con modelos similares como Faster R-CNN y SSD. Adicionalmente, se tomó en cuenta que la arquitectura YOLO es open source y está bastante bien documentada / explicada lo que facilitó su implementación por mi parte.

YOLO es una red convolucional de 24 capas convolucionales, 4 capas de max pooling y 2 capas densas como se muestra en la siguiente imagen:

![image](YOLO_ARCHITECTURE.png)

Por ultimo, un factor determinante para la selección de yolo fue el hecho de que maneja bastante bien la detección de objetos pequeños ya que usa un enfoque regresivo mediante el cual segmenta la imagen en multiples cuadrantes y busca predecir la bounding-box a diferentes escalas.

### Resultados:

Ya que no se está trabajando con una tarea de clasificación, en cambio de detección, se utilizó como métrica de evaluación general el Índice de Jaccard, la intersección entre la predicción y el verdadero valor sobre su union, es decir, busca evaluar que porcentaje del área predecida es correcto.

A continuación se dseplegan los resultados:

- Control (10 EPOCHS, ADAM, MSE):
    - Indice de Jaccard Promedio: ~0.175
- Control con data augmentation (10 EPOCHS, ADAM, MSE):
    - Indice de Jaccard Promedio: ~0.217
- YOLO (10 EPOCHS, ADAM, MSE):
    - Indice de Jaccard Promedio: ~0.786
    - Precision: ~0.909
    - Recall: ~0.870
- YOLO con data augmentation (10 EPOCHS, ADAM, MSE):
    - Indice de Jaccard Promedio: ~0.763
    - Precision: ~0.95
    - Recall: ~0.823
- YOLO con data augmentation (10 EPOCHS, ADAMW, MSE):
    - Indice de Jaccard Promedio: ~0.715
    - Precision: ~0.76
    - Recall: ~0.826
- YOLO con data augmentation (50 EPOCHS, ADAM, MSE):
    - Indice de Jaccard Promedio: ~0.764
    - Precision: ~0.846
    - Recall: ~0.956

Ya que el modelo YOLO tiene la opcion de no detectar una placa, se evaluó también la precision, recall y f1 score de la cantidad de detección respecto a la cantidad de placas en la imagen.

Claramente el mejor modelo fue el YOLO, especifciamente el primer modelo YOLO (10 EPOCHS, ADAM, MSE) el cual tuvo le mejor desempeño. Parece ser que el implementar data augmentation en el caso del modelo YOLO empeora su desempeño, auqnue bien podría ayudar con la generalización y el overfit, lo cual desfortunademente no se evaluó. Adicionalmente, el incremento en epocas de entrenamiento no parece tener un impacto muy significativo en el desmpeño del modelo. Y el mejor optimizador para la tarea es el ADAM, no ADAMW.

El mejor modelo, el YOLO (10 EPOCHS, ADAM, MSE) sin data augmentation tiene un índice de Jaccard promedio de 78.6% sobre las detecciones que se realizó. Y respecto a la cantidad de detecciones por imagen, tiene una precisión del 95%, un recall del 82.3%. Es decir detecta el 82.3% de las placas existentes. Y de la cantidad de placas predecidas, el 95% corresponden a la cantidad real por imagen.

##### Conclusión:

Nuevamente, el mejor modelo fue el modelo YOLO, You Only Look Once, re entrendo con el set de imagenes sin data augmentation a 10 epocas, con el optimizador ADAM y la función de perdiad Mean Squared Error. Estos fueron seleccionados ya que cuentan con el mejor desempeño, principalment en el indice de Jaccard el cual fue la métrica principal debido a la naturaleza del problema.

